In [1]:
import pandas as pd
import joblib
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, mean_squared_error

In [10]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')

In [11]:
O_E = OrdinalEncoder()
train[['id','Target']] = O_E.fit_transform(train[['id','Target']])
train.head(2)

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0.0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,2.0
1,1.0,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,0.0


In [12]:
duplicates = train[train.duplicated()]
print(duplicates)

Empty DataFrame
Columns: [id, Marital status, Application mode, Application order, Course, Daytime/evening attendance, Previous qualification, Previous qualification (grade), Nacionality, Mother's qualification, Father's qualification, Mother's occupation, Father's occupation, Admission grade, Displaced, Educational special needs, Debtor, Tuition fees up to date, Gender, Scholarship holder, Age at enrollment, International, Curricular units 1st sem (credited), Curricular units 1st sem (enrolled), Curricular units 1st sem (evaluations), Curricular units 1st sem (approved), Curricular units 1st sem (grade), Curricular units 1st sem (without evaluations), Curricular units 2nd sem (credited), Curricular units 2nd sem (enrolled), Curricular units 2nd sem (evaluations), Curricular units 2nd sem (approved), Curricular units 2nd sem (grade), Curricular units 2nd sem (without evaluations), Unemployment rate, Inflation rate, GDP, Target]
Index: []

[0 rows x 38 columns]


In [7]:
train.corr()['Target'].sort_values(ascending=False)

Target                                            1.000000
Curricular units 2nd sem (approved)               0.781452
Curricular units 1st sem (approved)               0.725490
Curricular units 2nd sem (grade)                  0.719036
Curricular units 1st sem (grade)                  0.661355
Tuition fees up to date                           0.415691
Scholarship holder                                0.394124
Curricular units 2nd sem (enrolled)               0.289165
Curricular units 1st sem (enrolled)               0.263657
Curricular units 2nd sem (evaluations)            0.214951
Admission grade                                   0.172880
Course                                            0.154208
Curricular units 1st sem (evaluations)            0.152398
Displaced                                         0.150066
Previous qualification (grade)                    0.138119
Application order                                 0.128394
Daytime/evening attendance                        0.1244

Drop clounms that has no correlation with indipendent variable
1 Educational special needs  
2 International
3 Nacionality
4 Unemployment rate
5 Curricular units 1st sem (credited) 
6 Curricular units 2nd sem (credited)
7 Mother's qualification
8 Previous qualification
9 Father's qualification
10 Father's occupation  
11 Inflation rate 

In [13]:
train = train.drop(columns=["Educational special needs", "International", "Nacionality", "Unemployment rate", "Curricular units 1st sem (credited)", 
                            "Curricular units 2nd sem (credited)", "Mother's qualification", "Previous qualification", "Father's qualification" 
                            ,"Father's occupation", "Inflation rate"])

In [19]:
test = test.drop(columns=["Educational special needs", "International", "Nacionality", "Unemployment rate", "Curricular units 1st sem (credited)", 
                            "Curricular units 2nd sem (credited)", "Mother's qualification", "Previous qualification", "Father's qualification" 
                            ,"Father's occupation", "Inflation rate"])

In [14]:
model = DecisionTreeClassifier(random_state=42)

In [15]:
features= train.drop(['id','Target'],axis=1)
target=train['Target']# store the labels .to_numpy()

In [16]:
model.fit(features, target)

DecisionTreeClassifier(random_state=42)

In [32]:
testset = test.drop('id' , axis =1)

In [37]:
pred = model.predict(testset)

In [ ]:
accuracy_score(testset,pred) 

In [38]:
predset = pd.DataFrame({'id': test['id'], 'Target': pred})
predset.head(5)

,id,Target
0,76518,0.0
1,76519,2.0
2,76520,2.0
3,76521,2.0
4,76522,1.0


In [39]:
def targetNames(col):
    if col == 0:
        return 'Dropout'
    if col == 1:
        return 'Enrolled'
    if col == 2:
        return 'Graduate'

prediction_df = pd.Series(pred).apply(targetNames)
preddf = pd.DataFrame({'id': test['id'], 'Target': prediction_df})
preddf.head(5)

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Graduate
4,76522,Enrolled


In [29]:
accuracy_score(sampledf,pred) 

0.0